In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

masoudnickparvar_brain_tumor_mri_dataset_path = kagglehub.dataset_download('masoudnickparvar/brain-tumor-mri-dataset')
dumbodhruvi_img_tensorflow2_default_1_path = kagglehub.model_download('dumbodhruvi/img/TensorFlow2/default/1')

print('Data source import complete.')



  0%|          | 0.00/4.13M [00:00<?, ?B/s]
 24%|██▍       | 1.00M/4.13M [00:00<00:01, 2.65MB/s]
100%|██████████| 4.13M/4.13M [00:00<00:00, 8.13MB/s]

Data source import complete.


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import os

tf.random.set_seed(42)
np.random.seed(42)
train_dir = '/kaggle/input/brain-tumor-mri-dataset/Training'
test_dir = '/kaggle/input/brain-tumor-mri-dataset/Testing'
classes = ['glioma', 'meningioma', 'pituitary', 'notumor']

for cls in classes:
    train_count = len(os.listdir(os.path.join(train_dir, cls)))
    test_count = len(os.listdir(os.path.join(test_dir, cls)))
    print(f"{cls}: {train_count} training images, {test_count} testing images")

glioma: 1321 training images, 300 testing images
meningioma: 1339 training images, 306 testing images
pituitary: 1457 training images, 300 testing images
notumor: 1595 training images, 405 testing images


In [5]:
# Data augmentation and preprocessing for training
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [6]:
# Training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    seed=42
)

Found 4571 images belonging to 4 classes.


In [7]:
# Validation generator
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    seed=42
)

Found 1141 images belonging to 4 classes.


In [8]:
# Test generator (no augmentation, only preprocessing)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 1311 images belonging to 4 classes.


In [9]:
# Build the ResNet152 model
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model initially

234698864/234698864 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [10]:
# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout to prevent overfitting
predictions = Dense(4, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [11]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

In [13]:
# Train the model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 645ms/step - accuracy: 0.5324 - loss: 1.2093

143/143 ━━━━━━━━━━━━━━━━━━━━ 162s 917ms/step - accuracy: 0.5334 - loss: 1.2070 - val_accuracy: 0.7143 - val_loss: 0.7873
Epoch 2/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step - accuracy: 0.8258 - loss: 0.4882

143/143 ━━━━━━━━━━━━━━━━━━━━ 145s 597ms/step - accuracy: 0.8258 - loss: 0.4880 - val_accuracy: 0.7572 - val_loss: 0.6234
Epoch 3/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 84s 586ms/step - accuracy: 0.8354 - loss: 0.4385 - val_accuracy: 0.7537 - val_loss: 0.5952
Epoch 4/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 465ms/step - accuracy: 0.8714 - loss: 0.3670

143/143 ━━━━━━━━━━━━━━━━━━━━ 86s 601ms/step - accuracy: 0.8714 - loss: 0.3670 - val_accuracy: 0.7844 - val_loss: 0.5509
Epoch 5/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step - accuracy: 0.8718 - loss: 0.3382

...

143/143 ━━━━━━━━━━━━━━━━━━━━ 89s 621ms/step - accuracy: 0.9301 - loss: 0.1913 - val_accuracy: 0.8685 - val_loss: 0.3514
Epoch 20/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 0.9302 - loss: 0.1905

143/143 ━━━━━━━━━━━━━━━━━━━━ 86s 599ms/step - accuracy: 0.9302 - loss: 0.1905 - val_accuracy: 0.8738 - val_loss: 0.3333
Epoch 21/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 0.9286 - loss: 0.1913

143/143 ━━━━━━━━━━━━━━━━━━━━ 85s 596ms/step - accuracy: 0.9286 - loss: 0.1912 - val_accuracy: 0.8755 - val_loss: 0.3268
Epoch 22/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 466ms/step - accuracy: 0.9308 - loss: 0.1856

143/143 ━━━━━━━━━━━━━━━━━━━━ 85s 595ms/step - accuracy: 0.9309 - loss: 0.1855 - val_accuracy: 0.8799 - val_loss: 0.3150
Epoch 23/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 84s 585ms/step - accuracy: 0.9151 - loss: 0.2114 - val_accuracy: 0.8712 - val_loss: 0.3476
Epoch 24/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 84s 585ms/step - accuracy: 0.9319 - loss: 0.1744 - val_accuracy: 0.8782 - val_loss: 0.3040
Epoch 25/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 466ms/step - accuracy: 0.9366 - loss: 0.1623

143/143 ━━━━━━━━━━━━━━━━━━━━ 87s 606ms/step - accuracy: 0.9365 - loss: 0.1624 - val_accuracy: 0.8878 - val_loss: 0.3133


In [14]:
# Load the best model
model.load_weights('best_model.h5')

In [15]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)
print(f'Test accuracy: {test_accuracy*100:.2f}%')

Test accuracy: 90.77%


In [17]:
# Fine-tuning if accuracy is below 90%
if test_accuracy < 0.95:
    print("Accuracy below 95%, proceeding with fine-tuning...")
    # Unfreeze the last 10 layers
    for layer in base_model.layers[-10:]:
        layer.trainable = True

    # Recompile with a lower learning rate
    model.compile(optimizer=Adam(learning_rate=1e-5),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Continue training
    history_fine = model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator,
        callbacks=[early_stopping, model_checkpoint],
        verbose=1
    )

    # Re-evaluate
    model.load_weights('best_model.h5')
    test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)
    print(f'Test accuracy after fine-tuning: {test_accuracy*100:.2f}%')

Accuracy below 90%, proceeding with fine-tuning...
Epoch 1/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 530ms/step - accuracy: 0.9342 - loss: 0.1809

143/143 ━━━━━━━━━━━━━━━━━━━━ 146s 801ms/step - accuracy: 0.9343 - loss: 0.1808 - val_accuracy: 0.8922 - val_loss: 0.3162
Epoch 2/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 85s 593ms/step - accuracy: 0.9416 - loss: 0.1541 - val_accuracy: 0.8878 - val_loss: 0.2869
Epoch 3/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 85s 593ms/step - accuracy: 0.9531 - loss: 0.1412 - val_accuracy: 0.8861 - val_loss: 0.2853
Epoch 4/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 85s 596ms/step - accuracy: 0.9523 - loss: 0.1416 - val_accuracy: 0.8922 - val_loss: 0.2822
Epoch 5/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step - accuracy: 0.9555 - loss: 0.1273

143/143 ━━━━━━━━━━━━━━━━━━━━ 91s 638ms/step - accuracy: 0.9555 - loss: 0.1274 - val_accuracy: 0.8975 - val_loss: 0.2573
Epoch 6/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 136s 595ms/step - accuracy: 0.9587 - loss: 0.1224 - val_accuracy: 0.8931 - val_loss: 0.2647
Epoch 7/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 85s 593ms/step - accuracy: 0.9626 - loss: 0.1138 - val_accuracy: 0.8896 - val_loss: 0.2784
Epoch 8/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 85s 593ms/step - accuracy: 0.9526 - loss: 0.1193 - val_accuracy: 0.8948 - val_loss: 0.2856
Epoch 9/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step - accuracy: 0.9583 - loss: 0.1086

143/143 ━━━━━━━━━━━━━━━━━━━━ 92s 643ms/step - accuracy: 0.9583 - loss: 0.1086 - val_accuracy: 0.9036 - val_loss: 0.2624
Epoch 10/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 470ms/step - accuracy: 0.9584 - loss: 0.1108

143/143 ━━━━━━━━━━━━━━━━━━━━ 92s 644ms/step - accuracy: 0.9584 - loss: 0.1108 - val_accuracy: 0.9115 - val_loss: 0.2550
Test accuracy after fine-tuning: 92.68%
